In [1]:
import sys
sys.path.append('./gym_dagsched/envs')

from copy import copy
import gym
import gym_dagsched.envs
import numpy as np
from dataclasses import asdict
import plotly.express as px
import pandas as pd
from visualization import make_gantt

env = gym.make(
    'dagsched-v0',
    max_jobs=10,        
    max_stages=8,      
    max_tasks=4,       
    n_worker_types=2,
    n_workers=5
)

In [2]:
obs = env.reset()
done = False

while not done:    
    frontier_stages = obs.get_frontier_stages()
    avail_workers = obs.find_available_workers()
    
    if len(avail_workers) == 0 or len(frontier_stages) == 0:
        if len(avail_workers) == 0:
            print('no available workers')
        else:
            print('frontier is empty')
        action = {
            'job_id': env.max_jobs,
            'stage_id': env.max_stages,
            'worker_type_counts': np.zeros(env.n_worker_types)
        }
    else:
        for stage in frontier_stages:
            for worker in avail_workers:
                if worker.compatible_with(stage):
                    first_stage = stage
        
        stage_id = first_stage.id_
        job_id = first_stage.job_id
        
        avail_counts = obs.get_avail_worker_counts(env.n_worker_types)
        
        stage = obs.jobs[job_id].stages[stage_id]
        worker_types = stage.compatible_worker_types()
        
        n_workers = 0
        worker_type_counts = np.zeros(env.n_worker_types, dtype=int)
        for worker_type in worker_types:
            if n_workers == stage.n_remaining_tasks:
                break
            avail_count = avail_counts[worker_type]
            request_count = int(min(stage.n_remaining_tasks - n_workers, avail_count))
            worker_type_counts[worker_type] = request_count
            n_workers += request_count
        
        print(f'n_workers={n_workers}')

        action = {
            'job_id': job_id,
            'stage_id': stage_id,
            'worker_type_counts': worker_type_counts
        }
    
    print()
    prev_obs = copy(obs)
    obs, _, done, _ = env.step(action)

frontier is empty

invalid action
2.5344230702533506: job arrival
n_workers=3

2.5344231128692627: nudge
n_workers=2

10.768179315787442: job arrival
no available workers

invalid action
31.7990779876709: task completion (0,2,0)
n_workers=1

31.7990779876709: task completion (0,2,1)
n_workers=1

31.7990779876709: task completion (0,2,2)
stage completion
n_workers=0

invalid action
37.906877649094696: job arrival
n_workers=1

39.61875915527344: task completion (0,1,0)
n_workers=1

39.61875915527344: task completion (0,1,1)
n_workers=1

57.174452175741976: job arrival
no available workers

invalid action
61.34402254915427: job arrival
no available workers

invalid action
62.782677587341524: job arrival
no available workers

invalid action
67.62466430664062: task completion (2,6,0)
n_workers=1

68.88341522216797: task completion (0,1,2)
n_workers=1

68.88341522216797: task completion (0,1,3)
stage completion
n_workers=1

69.3365478515625: task completion (2,6,1)
n_workers=1

69.3365478515

In [3]:
env.observation_space.contains(asdict(prev_obs))

True

In [5]:
fig_gantt, df_gantt = make_gantt(prev_obs)
fig_gantt.show()